In [2]:
# 1. Import Modules

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

from numpy import dot
from numpy.linalg import norm
from tqdm import tqdm
from paths import *
from functions import *

In [4]:
# 2. 학습, 평가 데이터 구분

A = pd.read_parquet(dataset_a)
B = pd.read_parquet(dataset_b)

A_0 = A[A["label"] == 0]
A_1 = A[A["label"] == 1]

B_0 = B[B["label"] == 0]
B_1 = B[B["label"] == 1]

A_test_0 = A_0.sample(frac=0.2, random_state=2024)
A_study_0 = A_0.drop(A_test_0.index)
A_test_1 = A_1.sample(frac=0.2, random_state=2024)
A_study_1 = A_1.drop(A_test_1.index)
A_test = pd.concat([A_test_0, A_test_1], ignore_index=True)
A_study = pd.concat([A_study_0, A_study_1], ignore_index=True)

B_test_0 = B_0.sample(frac=0.2, random_state=2024)
B_study_0 = B_0.drop(B_test_0.index)
B_test_1 = B_1.sample(frac=0.2, random_state=2024)
B_study_1 = B_1.drop(B_test_1.index)
B_test = pd.concat([B_test_0, B_test_1], ignore_index=True)
B_study = pd.concat([B_study_0, B_study_1], ignore_index=True)

createFolder(output_a)
A_test.to_parquet(test_a)
A_study.to_parquet(study_a)
createFolder(output_b)
B_test.to_parquet(test_b)
B_study.to_parquet(study_b)

In [3]:
# 3. Study Dataset에서 Label을 분리하고 N개 Sampling

n = 10000 # Sample Size

createFolder(samples_a)
createFolder(samples_b)

A_study = pd.read_parquet(study_a)
B_study = pd.read_parquet(study_b)
A_study = A_study[A_study["label"] == 0]
B_study = B_study[B_study["label"] == 0]

print("A_study : ", str(len(A_study.index)))
print("B_study : ", str(len(B_study.index)))

A_k = df_sampling(A_study, n, samples_a)
B_k = df_sampling(B_study, n, samples_b)

print("A_K : ", str(A_k))
print("B_K : ", str(B_k))

FileNotFoundError: [Errno 2] No such file or directory: '..\\03_Output\\A\\study.parquet'

In [4]:
# 5. Sample 별 Prototype Clustering
import multiprocessing as mp
from functions import *

createFolder(clusters_a)
createFolder(clusters_b)

params = []
sample_A = os.listdir(samples_a)
sample_B = os.listdir(samples_b)

for sample in sample_A:
    param = [sample, samples_a, clusters_a]
    params.append(param)
for sample in sample_B:
    param = [sample, samples_b, clusters_b]
    params.append(param)

with mp.Pool(8) as p:
    imap = p.imap(prototype_clustering, params)
    results = list(tqdm(imap, total = len(params), desc="진행도"))
p.join()
p.close()

print("DONE")

진행도: 100%|████████████████████████████████████████████████████████████████████| 272/272 [17:00:47<00:00, 225.18s/it]

DONE


In [4]:
# 6. Cluster에서 대표값 추출
import multiprocessing as mp
from tqdm import tqdm

createFolder(results_a)
createFolder(results_b)

params = []
samples_A = os.listdir(samples_a)
if (".ipynb_checkpoints" in samples_A): samples_A.remove(".ipynb_checkpoints")
samples_B = os.listdir(samples_b)
if (".ipynb_checkpoints" in samples_B): samples_B.remove("ipynb_checkpoints")

for sample in samples_A:
    param = [sample, clusters_a, results_a]
    params.append(param)
for sample in samples_B:
    param = [sample, clusters_b, results_b]
    params.append(param)
    
with mp.Pool(8) as p:
    imap = p.imap(sample_clustering, params)
    results = list(tqdm(imap, total = len(params), desc="진행도"))
p.join()
p.close()

# A
ks = os.listdir(results_a)
if (".ipynb_checkpoints" in ks): ks.remove(".ipynb_checkpoints")
if ("result" in ks): ks.remove("result")
os.makedirs(results_a + "result/", exist_ok=True)

for k in ks:
    ans = pd.DataFrame(columns=pd.read_csv(results_a + k + "\\1.parquet_result.csv").columns)
    results = os.listdir(results_a + k + "\\")
    if (".ipynb_checkpoints" in results): results.remove(".ipynb_checkpoints")
    for result in results:
        tmp = pd.read_csv(results_a + k + "\\" + result)
        ans = pd.concat([ans, tmp])
    ans.to_csv(results_a + "result\\" + str(k) + "_result.csv")
    print("Done ... ", str(k), "\t", len(ans.index))

# B
ks = os.listdir(results_b)
if (".ipynb_checkpoints" in ks): ks.remove(".ipynb_checkpoints")
if ("result" in ks): ks.remove("result")
os.makedirs(results_b + "result/", exist_ok=True)

for k in ks:
    ans = pd.DataFrame(columns=pd.read_csv(results_b + k + "\\1.parquet_result.csv").columns)
    results = os.listdir(results_b + k + "\\")
    if (".ipynb_checkpoints" in results): results.remove(".ipynb_checkpoints")
    for result in results:
        tmp = pd.read_csv(results_b + k + "\\" + result)
        ans = pd.concat([ans, tmp])
    ans.to_csv(results_b + "result\\" + str(k) + "_result.csv")
    print("Done ... ", str(k), "\t", len(ans.index))

print("Done")

진행도: 100%|████████████████████████████████████████████████████████████████████████| 272/272 [06:26<00:00,  1.42s/it]


Done ...  1 	 209518
Done ...  10 	 800137
Done ...  2 	 344414
Done ...  20 	 800137
Done ...  30 	 800137
Done ...  4 	 618483
Done ...  40 	 800137
Done ...  50 	 800137
Done ...  8 	 800137
Done ...  1 	 117181
Done ...  10 	 454997
Done ...  2 	 150680
Done ...  20 	 865618
Done ...  30 	 1291327
Done ...  4 	 222720
Done ...  40 	 1706276
Done ...  50 	 1818841
Done ...  8 	 375572
Done


In [6]:
# 6-1. ML Test (Before)
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from tqdm import tqdm
import os, time

# Malware
test = pd.read_parquet(test_a)
x_test = []
for i in tqdm(range(len(test.index))):
    x_test.append(char256(test.iloc[i].payload))
y_test = [int(x) for x in test.label.tolist()]

study = pd.read_parquet(study_a)
x_train = []
for i in tqdm(range(len(study.index))):
    x_train.append(char256(study.iloc[i].payload))
y_train = [int(x) for x in study.label.tolist()]

start = time.time()
model = RandomForestClassifier(n_estimators=100, random_state=0, n_jobs=-1)
model.fit(x_train, y_train)
prediction = model.predict(x_test)
end = time.time()

print("Before-A")
print("F1 Score : ", round(f1_score(y_test, prediction), 4))
print("Accurancy Score : ", round(accuracy_score(y_test, prediction), 4))
print("Precision Score : ", round(precision_score(y_test, prediction), 4))
print("Recall Score : ", round(recall_score(y_test, prediction), 4))
print("Run Time : ", end - start, "sec")

# Web
test = pd.read_parquet(test_b)
x_test = []
for i in tqdm(range(len(test.index))):
    x_test.append(char256(test.iloc[i].payload))
y_test = [int(x) for x in test.label.tolist()]

study = pd.read_parquet(study_b)
x_train = []
for i in tqdm(range(len(study.index))):
    x_train.append(char256(study.iloc[i].payload))
y_train = [int(x) for x in study.label.tolist()]

start = time.time()
model = RandomForestClassifier(n_estimators=100, random_state=0, n_jobs=-1)
model.fit(x_train, y_train)
prediction = model.predict(x_test)
end = time.time()

print("Before-B")
print("F1 Score : ", round(f1_score(y_test, prediction), 4))
print("Accurancy Score : ", round(accuracy_score(y_test, prediction), 4))
print("Precision Score : ", round(precision_score(y_test, prediction), 4))
print("Recall Score : ", round(recall_score(y_test, prediction), 4))
print("Run Time : ", end - start, "sec")

100%|████████████████████████████████████████████████████████████████████████| 894605/894605 [02:33<00:00, 5814.90it/s]


Before-A
F1 Score :  0.9811
Accurancy Score :  0.9989
Precision Score :  0.9749
Recall Score :  0.9874
Run Time :  47.34644269943237 sec


100%|██████████████████████████████████████████████████████████████████████| 1869338/1869338 [07:43<00:00, 4030.54it/s]


Before-B
F1 Score :  0.8312
Accurancy Score :  0.9954
Precision Score :  0.9329
Recall Score :  0.7494
Run Time :  120.83888864517212 sec


In [9]:
# 6-2. ML Test (After)
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from tqdm import tqdm
import os, time

# Malware
ans_total = {}
test = pd.read_parquet(test_a)
x_test = []
for i in tqdm(range(len(test.index))):
    x_test.append(char256(test.iloc[i].payload))
y_test = [int(x) for x in test.label.tolist()]

studys = os.listdir(results_a + "result\\")
if (".ipynb_checkpoints" in studys): studys.remove(".ipynb_checkpoints")
for path in studys:
    study = pd.read_parquet(study_a)
    study = study[study["label"] == 1]
    result = pd.read_csv(results_a + "result\\" + path)
    study = pd.concat([study, result])
    
    x_train = []
    for i in tqdm(range(len(study.index))):
        x_train.append(char256(study.iloc[i].payload))
    y_train = [int(x) for x in study.label.tolist()]
    
    start = time.time()
    model = RandomForestClassifier(n_estimators=100, random_state=0, n_jobs=-1)
    model.fit(x_train, y_train)
    prediction = model.predict(x_test)
    end = time.time()
    ans = [f1_score(y_test, prediction), accuracy_score(y_test, prediction), precision_score(y_test, prediction), recall_score(y_test, prediction)]
    name = path[:path.find('_')]
    ans_total[name] = ans
    print(f"A-After : {name}")
    print("F1 Score : ", round(f1_score(y_test, prediction), 4))
    print("Accurancy Score : ", round(accuracy_score(y_test, prediction), 4))
    print("Precision Score : ", round(precision_score(y_test, prediction), 4))
    print("Recall Score : ", round(recall_score(y_test, prediction), 4))
    print("Run Time : ", end - start, "sec")

print("==============================================")
# Web
ans_total = {}
test = pd.read_parquet(test_b)
x_test = []
for i in tqdm(range(len(test.index))):
    x_test.append(char256(test.iloc[i].payload))
y_test = [int(x) for x in test.label.tolist()]

studys = os.listdir(results_b + "result/")
if (".ipynb_checkpoints" in studys): studys.remove(".ipynb_checkpoints")
for path in studys:
    study = pd.read_parquet(study_b)
    study = study[study["label"] == 1]
    result = pd.read_csv(results_b + "result/" + path)
    study = pd.concat([study, result])
    
    x_train = []
    for i in tqdm(range(len(study.index))):
        x_train.append(char256(study.iloc[i].payload))
    y_train = [int(x) for x in study["label"].tolist()]
    
    start = time.time()
    model = RandomForestClassifier(n_estimators=100, random_state=0, n_jobs=-1)
    model.fit(x_train, y_train)
    prediction = model.predict(x_test)
    end = time.time()
    ans = [f1_score(y_test, prediction), accuracy_score(y_test, prediction), precision_score(y_test, prediction), recall_score(y_test, prediction)]
    name = path[:path.find('_')]
    ans_total[name] = ans
    print(f"B-After : {name}")
    print("F1 Score : ", round(f1_score(y_test, prediction), 4))
    print("Accurancy Score : ", round(accuracy_score(y_test, prediction), 4))
    print("Precision Score : ", round(precision_score(y_test, prediction), 4))
    print("Recall Score : ", round(recall_score(y_test, prediction), 4))
    print("Run Time : ", end - start, "sec")

100%|████████████████████████████████████████████████████████████████████████| 825187/825187 [02:11<00:00, 6262.21it/s]


A-After : 10
F1 Score :  0.9811
Accurancy Score :  0.9989
Precision Score :  0.9742
Recall Score :  0.9882
Run Time :  36.69229698181152 sec


100%|████████████████████████████████████████████████████████████████████████| 234568/234568 [00:34<00:00, 6775.16it/s]


A-After : 1
F1 Score :  0.9794
Accurancy Score :  0.9988
Precision Score :  0.9674
Recall Score :  0.9917
Run Time :  11.934755563735962 sec


100%|████████████████████████████████████████████████████████████████████████| 825187/825187 [02:11<00:00, 6266.87it/s]


...

B-After : 4
F1 Score :  0.7686
Accurancy Score :  0.9918
Precision Score :  0.6679
Recall Score :  0.905
Run Time :  16.0415940284729 sec


100%|██████████████████████████████████████████████████████████████████████| 1847063/1847063 [07:20<00:00, 4196.02it/s]


B-After : 50
F1 Score :  0.8331
Accurancy Score :  0.9954
Precision Score :  0.9263
Recall Score :  0.7569
Run Time :  126.9157452583313 sec


100%|████████████████████████████████████████████████████████████████████████| 403794/403794 [01:31<00:00, 4400.16it/s]


B-After : 8
F1 Score :  0.7885
Accurancy Score :  0.9928
Precision Score :  0.7119
Recall Score :  0.8835
Run Time :  24.138463258743286 sec
